<a href="https://colab.research.google.com/github/Komal77rao/Data-Eng-Modules/blob/main/index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to the Adapter Pattern

### Introduction

In the next lessons, we'll see how to use objects to better organize our code that makes requests to APIs.  As a first step, let's use our list of dictionaries from the API to construct a list of receipt objects.  Let's get started.

### APIs

First let's make a request for Max's Wine Dive.

In [1]:
import requests
response = requests.get("https://data.texas.gov/resource/naix-2893.json?location_name=MAX%27S%20WINE%20DIVE")
restaurant_receipts = response.json()

In [ ]:
restaurant_receipts[0]

{'taxpayer_number': '12727298569',
 'taxpayer_name': 'MWD AUSTIN DOWNTOWN, LLC',
 'taxpayer_address': '3 MEEKER LN',
 'taxpayer_city': 'COLORADO SPRINGS',
 'taxpayer_state': 'CO',
 'taxpayer_zip': '80921',
 'taxpayer_county': '0',
 'location_number': '1',
 'location_name': "MAX'S WINE DIVE",
 'location_address': '207 SAN JACINTO BLVD STE 200',
 'location_city': 'AUSTIN',
 'location_state': 'TX',
 'location_zip': '78701',
 'location_county': '227',
 'inside_outside_city_limits_code_y_n': 'Y',
 'tabc_permit_number': 'MB944126',
 'responsibility_begin_date_yyyymmdd': '2016-05-13T00:00:00.000',
 'responsibility_end_date_yyyymmdd': '2020-11-20T00:00:00.000',
 'obligation_end_date_yyyymmdd': '2017-03-31T00:00:00.000',
 'liquor_receipts': '28062',
 'wine_receipts': '77831',
 'beer_receipts': '14155',
 'cover_charge_receipts': '0',
 'total_receipts': '120048'}

### Our task

Now imagine that you are looking to analyze monthly revenue at Max's Wine Bar locations.  We would want relevant data from the dictionary organized into instances.  So refactor the code so that we still retrieve data from the api, but then we convert each of the dictionaries into instances and store them in a list.


In pursuing the above task, aim for the following:

1. Choose no more than five attributes and organize that information into a class so that we can turn each dictionary into an instance.
2. Only data you decide is relevant should be stored in your instances
3. Use the features of object orientation previously discussed -- __init__, custom methods -- where appropriate
4. You should be able to justify the decisions that you make

* Stretch goal

Instead of using object orientation with *only* the model layer -- say to create receipt instances -- move all of your code currently not in your model into a separate class.  



### The steps of refactoring

In [2]:
import requests

class Client:

  def get_response(self):
    response = requests.get("https://data.texas.gov/resource/naix-2893.json?location_name=MAX%27S%20WINE%20DIVE")
    restaurant_receipts = response.json()
    return restaurant_receipts

#location_name, total_receipts, location_city, location_state, location_county
class Receipt:
    class_dict = {'location_name':'name', 'total_receipts':'total','location_city':'city','location_state':'state','location_county':'county'}
    def __init__(self,**kwargs):
      self.__dict__ = {self.class_dict[k]:v for k,v in kwargs.items() if k in self.class_dict}

In [3]:
client = Client()
receipts_json = client.get_response()

In [4]:
receipt = Receipt(**receipts_json[0])

In [8]:
receipt.__dict__

{'name': "MAX'S WINE DIVE",
 'city': 'DALLAS',
 'state': 'TX',
 'county': '57',
 'total': '0'}

**Before you get started**, this is how to write refactored code.

1. Get your code working first, and then refactor some of the code, and ensure that the code is still working, then refactor more
    * This is called "Red, Green, Refactor".  
    * Red is the code in a broken state, green is getting the code to work, and refactoring is refactoring.

2. In refactoring the code think about:
    * Reject, Coerce, Act, Return
    * Write small code:
        * Lots of small objects is often better than a few large objects
        * Lots of small methods is better than a few large methods
    